# Tikhonov regularization, the conjugate gradient method and the L-curve
Consider the problem from lab 3. Another approach that can avoid the effects of semi-convergence is to solve instead of the least-squares problem
$$ \min_{\alpha} \frac{1}{2} \| A \alpha - g \|^2 $$
the regularized problem
$$ \min_{\alpha} \frac{1}{2} \| A \alpha - g \|^2 + \frac{\lambda}{2} \| \alpha \|^2 $$
with a regularization parameter $\lambda > 0$. It turns out that for a suitably chosen parameter $\lambda$ (depending on the noise level and the problem) the exact solution of the regularized problem (called [Tikhonov regularization](https://en.wikipedia.org/wiki/Tikhonov_regularization)) is a more accurate reconstruction than a solution of the unregularized problem (though there are typically better regularization alternatives). The optimality condition now reads $0 = A^t (A \alpha - g) + \lambda \alpha$. For the reconstruction problem described in lab 3, find a regularized solution for some choice of $\lambda$ using the conjugate gradient method and plot the relative error. Then, try to find a good choice of the parameter $\lambda$ with the L-curve method. For the L-curve you should not do a full reconstruction for each $\lambda$, but only a few iterations to save time. Compare with the Landweber solution.

In [101]:
import odl
import numpy as np
import matplotlib.pyplot as plt